In [1]:
!pip install -r requirements.txt
!pip uninstall -y tinarm
!pip install -e ../.

  Cloning https://github.com/TinArmEngineering/tinarm.git (to revision 0.1.81) to /private/var/folders/s5/09ndf2214qb_308ln1n6gfm40000gn/T/pip-req-build-120yjgyd
  Running command git clone --filter=blob:none --quiet https://github.com/TinArmEngineering/tinarm.git /private/var/folders/s5/09ndf2214qb_308ln1n6gfm40000gn/T/pip-req-build-120yjgyd
  Running command git checkout -q 95326bba5f50040358d1e4c697986c1457bd751f
  Resolved https://github.com/TinArmEngineering/tinarm.git to commit 95326bba5f50040358d1e4c697986c1457bd751f
  Preparing metadata (setup.py) ... done
  Created wheel for tinarm: filename=tinarm-0.1-py3-none-any.whl size=7253 sha256=61b4658d8a9059004efcceff2eb4ca6865fb765dd08affd1bf0667e15de0a7e7
  Stored in directory: /private/var/folders/s5/09ndf2214qb_308ln1n6gfm40000gn/T/pip-ephem-wheel-cache-9lze4osi/wheels/b5/3e/f1/78975b6840059f1f22cfabad54abc26bd24ff9224d419ec7e4
Successfully built tinarm
  Attempting uninstall: tinarm
    Found existing installation: tinarm 0.1
   

In [2]:
import matplotlib.pyplot as plt
import pint
# from tinarm import NameQuantityPair
# from tinarm import Quantity 
from tinarm import Job
from tinarm import Machine
from tinarm.api import JOB_STATUS
from tinarm import Api

import logging
import time
import requests
import uuid
import random
import yaml
import pandas as pd
import pint_pandas


In [3]:

LOGGING_LEVEL = logging.INFO
STATUS_JOB = {value:key for key,value in JOB_STATUS.items()}

In [4]:

### Configure Logging
logger = logging.getLogger()
logger.setLevel(LOGGING_LEVEL)
logger.info(f"tinarm version {tinarm.__version__}")
logger.info(f"pint_pandas version {pint_pandas.__version__}")

q = pint_pandas.PintType.ureg
q.setup_matplotlib()

2024-03-11 11:20:16,573 - NoJobId - INFO - Martins-MacBook-Air.local - 1683099798.py-><module>() - tinarm version 0.1
2024-03-11 11:20:16,573 - NoJobId - INFO - Martins-MacBook-Air.local - 1683099798.py-><module>() - pint_pandas version 0.5


Log in to Tin Arm Engineering's machine solver, and under profile, retrieve your API key. 
Do not commit your API key to a repository, and consider it like a password.  A good way to keep it out of this code base is to use a configuation file, or environment variable. We will want other things cofigured too, so let's use a yaml file for convenience. 

create a `configurations.yaml` file in this directory with the content.
```yaml
api_key: 668952058c490d0a296da0abb966****
root_url: http://server-go:4300
```
But of course replace the number with your api key

You can then add that file to your `.gitignore` file by executing ```echo 'configurations.yaml' >> .gitignore```

```
!echo 'configurations.yaml' >> .gitignore
```

In [14]:
with open("configurations.yaml", "r") as f:
    config = yaml.safe_load(f)

# Stator

In [15]:
stator_parameters = {
    "slot_liner_thikness": 300 * q.um,
    "stator_bore": 8.20 * q.cm,
    "tooth_tip_depth": 1.5 * q.mm,
    "slot_opening": 1.5 * q.mm,
    "tooth_width": 9.8 * q.mm,
    "stator_outer_diameter": 0.136 * q.m,
    "back_iron_thickness": 5.5 * q.mm,
    "stator_internal_radius": 500 * q.um,
    "number_slots": 12 * q.count,
    "tooth_tip_angle": 70 * q.degrees
    }

air_gap_length = 1 * q.mm

## Rotor
Surface mounted Breadloaf magnets

In [16]:
rotor_parameters = {
    "rotor_od": stator_parameters["stator_bore"] - 2 * air_gap_length,
    "rotor_bore": 40 * q.mm,
    "banding_thickness": 0.5 * q.mm,
    "number_poles": 10 * q.count,
    "magnet_thickness": 4.5 * q.millimeter,
    "magnet_pole_arc": 150 * q.degrees,
    "magnet_inset": 0.25 * q.millimeter
    }

## Simulation Parameters

In [17]:
simulation_parameters = {
       "samples_per_electrical_period": 180 * q.count/q.turn,
        "timestep_intervals": 180 * q.count,
        "active_length": 65 * q.mm}


In [18]:
winding_parameters = {
    "symmetry": 2 * q.count,
    "number_phases": 3 * q.count,
    "number_layers": 2 * q.count,
    "coil_span": 1 * q.count,
    "turns_per_coil": 43 * q.count,
    "empty_slots": 0 * q.count,
    "fill_factor": 42 * q.percent
    }

## Operating Point

In [19]:
op_open_cct = {
    "simulated_speed": 2060 * q.rpm,
    "current_density": 0 * q.A / q.mm ** 2,
    "current_angle": 0 * q.degrees
    }

op_nominal = {
    'current_density': 6.23 * q.A * q.mm**-2,
    'current_angle': 255 * q.degrees,
    "simulated_speed": 2060 * q.rpm
    }

In [20]:
# class Api:
#     """
#     The TAE User API
#     """

#     def __init__(self, root_url, api_key, org_id=None):
#         """
#         Initialize the API
#         """
#         self._root_url = root_url
#         self._api_key = api_key
#         self._org_id = org_id
#         self._node_id = None

#         logger.info(f"root_url: {self._root_url}")

#     def get_job(self, job_id):
#         """
#         Get a job from the TAE API
#         """
#         response = requests.get(
#             url=f"{self._root_url}/jobs/{job_id}?apikey={self._api_key}",
#         )
#         response.raise_for_status()
#         return response.json()
    
#     def create_job(self, job):
#         """
#         Create a job for the TAE API
#         """
#         response = requests.post(
#             url=f"{self._root_url}/jobs/?apikey={self._api_key}&org_id={self._org_id}",
#             json=job.to_api()
#         )
#         response.raise_for_status()
#         if response.status_code == 200:
#             job.id = response.json()["id"]
#         return response.json()
    
#     def update_job_status(self, job_id, status):
#         """
#         Update a job status
#         """
#         url = f"{self._root_url}/jobs/{job_id}/status/{status}?node_id={self._node_id}&apikey={self._api_key}"
#         logger.info(f"Updating job status: {url}")

#         response = requests.put(url=url)
#         return response.json()
    
    

In [21]:
api = Api(config["root_url"], config["api_key"], config["org_id"])

2024-03-11 11:26:42,261 - NoJobId - INFO - Martins-MacBook-Air.local - 3503141280.py->__init__() - root_url: https://api.build.tinarmengineering.com


In [22]:
m1 = Machine(stator_parameters, rotor_parameters, winding_parameters)

j1 = Job(m1, op_open_cct, simulation_parameters)
j1_result = api.create_job(j1)



In [23]:

j2 = Job(m1, op_nominal, simulation_parameters)
j2_result = api.create_job(j2)

In [24]:
assert j1.id != j2.id


In [25]:
j1_result = api.update_job_status(j1.id, JOB_STATUS['QueuedForMeshing'])
j2_result = api.update_job_status(j2.id, JOB_STATUS['QueuedForMeshing'])

while STATUS_JOB[api.get_job(j1.id)['status']] != 'Complete' or STATUS_JOB[api.get_job(j2.id)['status']] != 'Complete':
    print("job 1 {0} \t job 2 {1}".format(STATUS_JOB[api.get_job(j1.id)['status']], STATUS_JOB[api.get_job(j2.id)['status']] ))
    time.sleep(10)
time.sleep(1)

2024-03-11 11:26:58,538 - NoJobId - INFO - Martins-MacBook-Air.local - 3503141280.py->update_job_status() - Updating job status: https://api.build.tinarmengineering.com/jobs/65eedc64d0dc835ca46bf759/status/10?node_id=None&apikey=f3655d31edd9a0a90fd00d5064c8bfb2
2024-03-11 11:26:58,834 - NoJobId - INFO - Martins-MacBook-Air.local - 3503141280.py->update_job_status() - Updating job status: https://api.build.tinarmengineering.com/jobs/65eedc6fd0dc835ca46bf75a/status/10?node_id=None&apikey=f3655d31edd9a0a90fd00d5064c8bfb2
job 1 Meshing 	 job 2 Meshing
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solv

In [ ]:
j1_result = api.get_job(j1.id)
for artifact in j1_result['artifacts']:
    print(artifact['type'], artifact['url'], artifact['id'])

In [ ]:
j1_result_data = [art for art in j1_result['artifacts'] if art['type']=='RESULT_DATA'][0]
j1_df = pd.read_csv(j1_result_data['url'], header=[0, 1], index_col=[0, 1]).pint.quantify(level=-1)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(j1_df['angles'], j1_df['Back_EMF_0'], label='Back_EMF_0')
ax.plot(j1_df['angles'], j1_df['Back_EMF_1'], label='Back_EMF_1')
ax.plot(j1_df['angles'], j1_df['Back_EMF_2'], label='Back_EMF_2')

In [ ]:
j2_result = api.get_job(j2.id)
for artifact in j2_result['artifacts']:
    print(artifact['type'], artifact['url'], artifact['id'])
time.sleep(1)    

In [ ]:
j2_result_data = [art for art in j2_result['artifacts'] if art['type']=='RESULT_DATA'][0]
j2_df = pd.read_csv(j2_result_data['url'], header=[0, 1], index_col=[0, 1]).pint.quantify(level=-1)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
l1, = ax.plot(j1_df['angles'], j1_df['Back_EMF_0'], label='Back_EMF_0', linestyle='--')
l2, = ax.plot(j1_df['angles'], j1_df['Back_EMF_1'], label='Back_EMF_1', linestyle='--',)
l3, = ax.plot(j1_df['angles'], j1_df['Back_EMF_2'], label='Back_EMF_2', linestyle='--')
ax.plot(j2_df['angles'], j2_df['Back_EMF_0'], label='nominal Back_EMF_0', color=l1.get_color())
ax.plot(j2_df['angles'], j2_df['Back_EMF_1'], label='nominal Back_EMF_1', color=l2.get_color())
ax.plot(j2_df['angles'], j2_df['Back_EMF_2'], label='nominal Back_EMF_2', color=l3.get_color())
ax.legend()

In [ ]:

fig = plt.figure(2)
ax = fig.add_subplot(111)
l1, = ax.plot(j1_df['angles'], j1_df['Scaled Torque'], label='open cct Scaled Torque')
l2, = ax.plot(j2_df['angles'], j2_df['Scaled Torque'], label='nominal Scaled Torque')
mean_torque = j2_df['Scaled Torque'].mean()
ax.axhline( mean_torque, label=f'Scaled Torque {mean_torque:2.4~P}', linestyle='--')
target_torque = 24.26 * q.N * q.m
ax.axhline( target_torque, label=f'Target {target_torque:2.4~P}', color='k', linestyle='--')

ax.legend()

In [ ]:
m1_df = pd.read_csv("Machine_Back_emfs_Machine_1.csv", skiprows=1, header=[0, 1], index_col=[0]).pint.quantify(level=-1)
m1_df.head()


In [ ]:
m1_df['Angles electrical'] = pint_pandas.PintArray(m1_df.index * m1.rotor['number_poles'] / 2, dtype="pint[degree]")
m1_df.dtypes

In [ ]:
fig = plt.figure(3)
ax = fig.add_subplot(111)
l1, = ax.plot(j1_df['angles'], -j1_df['Back_EMF_0'], label='Back_EMF_0', linestyle='--')
l2, = ax.plot(j1_df['angles'], -j1_df['Back_EMF_1'], label='Back_EMF_1', linestyle='--',)
l3, = ax.plot(j1_df['angles'], -j1_df['Back_EMF_2'], label='Back_EMF_2', linestyle='--')
ax.plot(m1_df['Angles electrical'].pint.to("radians"), m1_df['Phase A'], label='Phase A', color=l1.get_color())
ax.plot(m1_df['Angles electrical'].pint.to("radians"), m1_df['Phase B'], label='Phase B', color=l2.get_color())
ax.plot(m1_df['Angles electrical'].pint.to("radians"), m1_df['Phase C'], label='Phase C', color=l3.get_color())


In [ ]:
df_m1_phvol = pd.read_csv("m1_phase_voltage_nominal.csv",  header=[0, 1])
df_m1_phvol.head()

In [ ]:
from numpy import pi
fig = plt.figure()
ax = fig.add_subplot(111)
l1, =ax.plot(df_m1_phvol["Va"]["X"]*pi*5/180, -df_m1_phvol["Va"]["Y"],linestyle='--')
l2, =ax.plot(df_m1_phvol["Vb"]["X"]*pi*5/180, -df_m1_phvol["Vb"]["Y"],linestyle='--')
l3, =ax.plot(df_m1_phvol["Vc"]["X"]*pi*5/180, -df_m1_phvol["Vc"]["Y"],linestyle='--')
ax.plot(j2_df['angles'], j2_df['Back_EMF_0'], label='nominal Back_EMF_0', color=l1.get_color())
ax.plot(j2_df['angles'], j2_df['Back_EMF_1'], label='nominal Back_EMF_1', color=l2.get_color())
ax.plot(j2_df['angles'], j2_df['Back_EMF_2'], label='nominal Back_EMF_2', color=l3.get_color())
ax.legend()

In [ ]:
df_m1_phvol["Va"]["X"]*(pi*5/180)

In [ ]:
df_m1_torque = pd.read_csv("M1AverageTorqueNominal.csv", names=["Angles", "Torque"], header=0)
df_m1_torque.head()

In [ ]:
fig = plt.figure(2)
ax = fig.add_subplot(111)
l1, = ax.plot(j1_df['angles'], j1_df['Scaled Torque'], label='open cct Scaled Torque')
l2, = ax.plot(j2_df['angles'], j2_df['Scaled Torque'], label='nominal Scaled Torque')
mean_torque = j2_df['Scaled Torque'].mean()
ax.axhline( mean_torque, label=f'Scaled Torque {mean_torque:2.4~P}', linestyle='--')
target_torque = 24.26 * q.N * q.m
ax.axhline( target_torque, label=f'Target {target_torque:2.4~P}', color='k', linestyle='--')
ax.plot(df_m1_torque["Angles"]*pi*5/180, df_m1_torque["Torque"], label='nominal Torque')
ax.legend()